In [3]:
"""
This file reads the monthly success rates from `/results/monthly/`, generates the yearly succuess rates, 
and then save them to `/results/annually/`
"""

from pandas import DataFrame, read_csv, Series
from datetime import datetime

# the input parameters
INPUT_PATH_PREFIX = "./results/monthly/"   
VERSIONS = ['Version3C', 'Version4A']
AQUA = "Aqua"
TERRA = "Terra"
TISA = "TISA"
TYPES = [AQUA, TERRA, TISA]

DATE_STR_FMT = "%b-%y"  # time format string, e.g, "Jan-21"

# ouput parameters
OUTPUT_PATH_PREFIX = "./results/annually/"

def convert_timestr_to_timeobj(time_str, fmt_str):
    """
    Convert a time string to a time object in Python.
    Note that when Python calling `read_csv` and reading `Jan-21`, it does not know 
    this string is a time. We need to mannually convert it to a time object in Python.
    """
    return datetime.strptime(time_str, fmt_str)


def convert_timestr_column_to_timeobj_column(timestr_column, fmt_str):
    """
    Convert a column of time strings to a column of time objects.
    """
    timeobj_column = []
    for t in timestr_column:
        timeobj_column.append(convert_timestr_to_timeobj(t, fmt_str))
    return timeobj_column


def load_one_column(table, column_name, index_name):
    return Series(table[column_name].values, index=index_name)


def save_to_csv(path_name_str, data):
    """
    Save a csv file with content as :param data to a file named as :param path_name_str
    """
    df = DataFrame(data)
    df.to_csv(path_name_str, index=False)


for type_str in TYPES:
    for version_str in VERSIONS:
        file_name_str = INPUT_PATH_PREFIX + type_str + '_' + version_str + '_SR_by_month.csv'
        
        df = read_csv(file_name_str)
        
        index_column = convert_timestr_column_to_timeobj_column(df['Month'].values, DATE_STR_FMT)
        annual_sr_1 = load_one_column(df, df.iloc[:, 1].name, index_column).resample('Y').mean()
        annual_sr_2 = load_one_column(df, df.iloc[:, 2].name, index_column).resample('Y').mean()
        
        data = {
            'Year': annual_sr_1.index.strftime('%Y'),   # reformat time str to year solely, e.g "2021"
            df.iloc[:, 1].name: annual_sr_1,
            df.iloc[:, 2].name: annual_sr_2
        }
        
        save_file_name_str = OUTPUT_PATH_PREFIX + type_str + '_' + version_str + '_SR_by_year.csv'  # create folder first
        save_to_csv(save_file_name_str, data)


NameError: name 'AQUA' is not defined